# Trucks only problem #

Install necessary packages

In [30]:
from gurobipy import Model,GRB,LinExpr,quicksum
import numpy as np
from scipy.spatial import distance
import os
import socket
from load_dataset import Dataset

For Ugo's laptop

In [31]:
# Define the node name or another identifier of your laptop
my_laptop_node = 'Ugos-MacBook-Pro.local'

# Get the current system's node name using socket.gethostname()
current_node = socket.gethostname()

if current_node == my_laptop_node:
    # Set the environment variable for Gurobi license file
    os.environ["GRB_LICENSE_FILE"] = "/Users/ugomunzi/gurobi/licenses/gurobi.lic"
    print("Gurobi license path set for Ugo's MacBook Pro.")
else:
    print("Not Ugo's MacBook Pro, using default or no specific license settings.")

Not Ugo's MacBook Pro, using default or no specific license settings.


Define model parametres

In [32]:
## MODEL PARAMETERS ##
W_T = 1500 #empty weight truck [kg]
Q_T = 1000 #load capacity of trucks [kg]
#W_D = 25 #empty weight drone [kg]
#Q_D = 5 #load capacity of drones [kg]
C_T = 25 #travel cost of trucks per unit distance [monetary unit/km]
#C_D = 1 #travel cost of drones per unit distance [monetary unit/km]
C_B = 500 #basis cost of using a truck equipped with a drone [monetary unit]
#E = 0.5 #maximum endurance of empty drones [hours]
S_T = 60 #average travel speed of the trucks [km/h]
#S_D = 65 #average travel speed of the drones [km/h]

Define Big M constant

In [33]:
M = 500 #big M constant for big M method

Load Dataset using load_dataset.py

In [34]:
## LOAD DATASET ##
current_dir = os.getcwd()
# Select which data folder to use
data_subfolder = '0.3'
data_subfoldercopy = '0.3_copy'
data_num_nodes = '40'
data_area = '20'

data_file_name = f'{data_num_nodes}_{data_area}_{data_subfoldercopy}'
dataset_path = f'dataset/{data_subfolder}/{data_file_name}.txt'
output_file_path = os.path.join(current_dir, data_file_name + '_solution.sol')#used to save solution file

dataset = Dataset(dataset_path)



Pre-processing

In [35]:
## FUNCTIONS ##
def get_manhattan_distance(data):
    """
    Returns a dictionary with manhattan distances between all nodes in dataset
    """
    distance_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            distance_dict[node1, node2] = distance.cityblock([data[node1]['X'], data[node1]['Y']], [data[node2]['X'], data[node2]['Y']])
    return distance_dict

def get_time_dict(data, S_T, distance_dict):
    """
    Returns a dictionary with travel times between all nodes in dataset
    """
    time_dict = {}
    for node1 in data.keys():
        for node2 in data.keys():
            time_dict[node1, node2] = distance_dict[node1, node2] / S_T
    return time_dict


num_trucks = 2
truck_distance_dict = get_manhattan_distance(dataset.data)
truck_time_dict = get_time_dict(dataset.data, S_T, truck_distance_dict)

#definitions of N_0, N and N_plus follow from paper
N = list(dataset.data.keys()) #set of nodes with depot at start
N_customers = N.copy()
N_customers.remove('D0')
Tr = [f'Tr{i}' for i in range(1, num_trucks+1)] #set of trucks

Define the model

In [36]:
# Define your data here
# Assume distance_dict, C_T, C_B, N, V, N_customers, time_dict, M, Q_T are all defined as per your problem context

# Create a new model
model = Model("Truck_Routing")

# Define decision variables
x = model.addVars(Tr, [(i, j) for i in N for j in N if i != j], lb=0, ub=1, vtype=GRB.BINARY, name='x')
y = model.addVars(Tr, lb=0, ub=1, vtype=GRB.BINARY, name='y')
t = model.addVars(Tr, N, lb=0, vtype=GRB.CONTINUOUS, name='t')
t_max = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name='t_max') #used for the minimising the max delivery time (find max time of all trucks, not each individual truck)

# Objective 1: Cost both due to transportation and base cost of using truck if active)
cost_obj = quicksum(C_T * truck_distance_dict[i,j] * x[truck,i,j] for i in N for j in N if i != j for truck in Tr) + \
           quicksum(C_B * y[truck] for truck in Tr)
# Objective 2: environmental_obj is distance[i,j] * Weight* x[v,i,j] for all v,i,j (i.e. energy consumption)
environmental_obj = quicksum(truck_distance_dict[i,j] * W_T * x[truck,i,j] for i in N for j in N if i != j for truck in Tr)
# Objective 3: minimise max delivery time for each truck
time_obj = t_max

obj = cost_obj + environmental_obj + time_obj
model.setObjective(obj, GRB.MINIMIZE)

model.update()

Constraint 1: Each customer is visited by exactly one truck

In [37]:
# Constraint 1: Each customer is visited by exactly one truck

# Each customer is visited by exactly one vehicle
constraints = {}
# Loop over each customer
for customer in N_customers:
    # Initialize the sum for the current customer
    sum_for_current_customer = 0

    # Loop over each truck
    for truck in Tr:
        # Loop over each node
        for node in N:
            # Skip if customer is equal to node
            if customer != node:
                # Add the variable to the sum
                sum_for_current_customer += x[truck, node, customer]

    # The sum for the current customer must be equal to 1
    constraints[customer] = model.addConstr(sum_for_current_customer == 1, name=f'Customer_{customer}_visited_once')


Constraint 2: Each depot must be visited exactly once

In [38]:
# Constraint 2: Each depot must be visited exactly once

# Each truck must leave the depot
# y - active
# 'D0' - depot
# Loop over each truck
for truck in Tr:
    sum_for_current_vehicle = quicksum(x[truck, 'D0', customer] for customer in N_customers)
    model.addConstr(sum_for_current_vehicle == y[truck], name=f'Truck_leaves_depot_{truck}')


Constraint 3: Each vehicle arrives at depot if active

In [39]:

# Constraint 3: Each truck arrives at depot if active : TRUCKS

# Each truck must return to the depot
# Loop over each truck
for truck in Tr:
    sum_for_current_vehicle = quicksum(x[truck, customer, 'D0'] for customer in N_customers)
    model.addConstr(sum_for_current_vehicle == y[truck], name=f'Truck_returns_to_depot_{truck}')


Constraint 4: If a vehicle arrives at a customer node it must also leave

In [40]:
# Constraint 4: If a truck arrives at a customer node it must also leave

# If a truck visits a customer, it must leave the customer
for truck in Tr:
    for node in N_customers:
        model.addConstr(
            quicksum(x[truck, node, j] for j in N if j != node) == 
            quicksum(x[truck, j, node] for j in N if j != node),
            name=f'Flow_balance_{truck}_{node}'
        )


Constraint 5: Time at a node is equal or larger than time at previous nodes plus travel time (or irrelevant). Eliminates need for subtour constraints.

In [41]:
'''
#Constraint 5: Time at a node is equal or larger than time at previous nodes plus travel time (or irrelevant). Eliminates need for subtour constraints.
# Define a large constant M for the big-M method : TRUCKS
'''
M_subtour = 60000000  # Make sure M is larger than the maximum possible travel time

# Add time constraints for all vehicles, nodes, and customers
for truck in Tr:
    for node in N:
        for customer in N:
            if node != customer:
                model.addConstr(
                    t[truck, customer] >= t[truck, node] + truck_time_dict[(node, customer)] - M_subtour * (1 - x[truck, node, customer]),
                    name=f'Time_{truck}_{node}_{customer}'
                )

Constraint 6: Payloads

In [42]:
# Constraint 6: Payloads : TRUCKS
# The total payload delivered to the customer must be less or equal to the truck load capacity Q_T
for truck in Tr:
    model.addConstr(quicksum(dataset.data[i]['Demand'] * x[truck, i, j] for i in N for j in N if i != j) <= Q_T, 
                    name=f'Payload_{truck}')

Constraint 7 : link y variable to x variable.

In [43]:
# Constraint 7: Link y variable to x variable : TRUCKS
#if any link in x (for each truck) is active -> y = 1
# can do this by checking if each truck leaves the depot (all trucks must leave depot if active)

for truck in Tr:
    model.addConstr(y[truck] == quicksum(x[truck, 'D0', i] for i in N_customers), name=f'Link_y{truck}_to_x_{truck}')


Constraint 8: update time variables.

In [44]:
# Constraint 8: Update time variable : TRUCKS
# Loop over each truck
for truck in Tr:
    # Loop over each customer
    for customer in N_customers:
        # Initialize the sum for the current customer
        sum_for_current_customer = 0

        # Loop over each node
        for node in N:
            # Skip the current customer
            if node != customer:
                # Add the time at which the truck leaves the node plus the travel time from the node to the customer,
                # multiplied by the decision variable indicating whether the truck travels from the node to the customer,
                # to the sum for the current customer
                sum_for_current_customer += (t[truck, node] + truck_time_dict[(node, customer)]) * x[truck, node, customer]

        # Add a constraint to the model that the time at which the truck arrives at the customer is equal to the sum for the current customer
        model.addConstr(t[truck, customer] == sum_for_current_customer, name=f'Update_time_{truck}_{customer}')


Constraint 9: update max delivery time variable.

In [45]:

# Constraint 9: Update max delivery time variable
for truck in Tr:
    for customer in N_customers:
        # Add a constraint to the model that the maximum delivery time is greater than or equal to the delivery time to the customer for each vehicle
        model.addConstr(t_max >= t[truck, customer], name=f'Update_max_delivery_time_{truck}_{customer}')

Update, tune and run the model

In [46]:
# Update the model to integrate new variables
model.update()

# Write model to a file
model.write('TruckonlySimple.lp')

# Tune solver parameters
model.tune()

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print('Optimal objective: %g' % model.objVal)
    for v in model.getVars():
        if v.x > 0:
            print('%s: %g' % (v.varName, v.x))
else:
    print('Optimization was stopped with status %d' % model.status)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Start tuning.

Solving model using baseline parameter set with TimeLimit=3600s

-------------------------------------------------------------------------------

Testing candidate parameter set 1...

	Default parameters

Solving Truck_Routing with random seed #1 ... MIP gap - (infeasible)
Solving Truck_Routing with random seed #2 ... MIP gap - (infeasible)
Solving Truck_Routing with random seed #3 ... MIP gap - (infeasible)

Summary candidate parameter set 1 
 # Name              0        1        2      Avg  Std Dev      Max
 0 Truck_Rou    0.34s#   0.33s#   0.27s#    0.31s     0.03    0.34s

Setting total tuning time limit to 9s
(set the TuneTimeLimit parameter to choose a different value)

----------------------------------------------------

Post-processing

In [47]:
# Extract and store the solution
solution = {var.varName: var.x for var in model.getVars()}

# Extract active trucks
active_trucks = [v for v in V if 'T' in v and solution[f'y[{v}]'] >= 0.99]

# Extract routes for active trucks
active_routes_truck = {}
for truck in active_trucks:
    active_routes_truck[truck] = []
    for node_from in N:
        for node_to in N:
            if node_from != node_to and solution.get(f'x[{truck},{node_from},{node_to}]', 0) >= 0.99:
                active_routes_truck[truck].append((node_from, node_to))

print('Active routes for trucks:', active_routes_truck)

AttributeError: Unable to retrieve attribute 'x'